# Demo Pipeline

In this workbook we will be demonstrating how to use streamsight to create
a pipeline to evaluate RecSys algorithms. Please refer to [demo.ipynb](demo.ipynb)
if you have not as it outlines how the pipeline works under the hood.

We will use Amazon movie data to show case the pipeline and some of the common
methods that you can call to evaluate your RecSys algorithms.

To start off, we will set the k+100 value to be 10 first. This will mean that
for any of the top K metric or algorithm, we will only consider the top 10
recommendations.

In [1]:
k = 10

## Load

We will load the dataset of choice and instantiate the setting that we want
to evaluate the algorithms on. We will use the sliding window setting for this
demo to show case the results of the evaluation.

Similarly to the demo, we can specify a range of parametres to create different
window sizes.

In [2]:
from streamsight.datasets import AmazonMovieDataset
from streamsight.settings import SlidingWindowSetting

dataset = AmazonMovieDataset(use_default_filters=False)
data = dataset.load()
setting_sliding = SlidingWindowSetting(
    background_t=1530000000,
    window_size=60 * 60 * 24 * 30, # day times N
    n_seq_data=1,
    top_K=k
)
setting_sliding.split(data)


INFO     - streamsight.datasets.base - AmazonMovieDataset is loading dataset...
INFO     - streamsight.datasets.base - AmazonMovieDataset dataset loaded - Took 12.9s


4it [00:01,  3.96it/s]                       

INFO     - streamsight.settings.sliding_window_setting - Finished split with window size 2592000 seconds. Number of splits: 4 in total.


## Evaluate

The evaluation of the algorithm will be abstracted by the pipeline. To create
the pipeline a builder class is used. This is the recommended way to create
pipelines as it allows for easy modification of the pipeline and easy
reproduction of the pipeline.

Adding of algorithm and metric can be done as shown below. Once the builder is
set up, the pipeline can will be returned by calling the `build` method.

Running the pipeline can be done via the `run` method. To run the pipeline in
step, the `run_step` method can be used. This will run the pipeline step by
step.

In [3]:
from streamsight.evaluators import EvaluatorPipelineBuilder

builder = EvaluatorPipelineBuilder(ignore_unknown_item=True,
                           ignore_unknown_user=True)
builder.add_setting(setting_sliding)
builder.set_metric_K(k)
builder.add_algorithm("ItemKNNStatic", {"K": k})
builder.add_algorithm("ItemKNNRolling", {"K": k})
builder.add_algorithm("ItemKNNIncremental", {"K": k})
builder.add_algorithm("Popularity", {"K": k})

builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
evaluator = builder.build()

evaluator.run()

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 1: Preparing the evaluator...


  0%|          | 0/4 [00:00<?, ?it/s]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


 25%|██▌       | 1/4 [00:47<02:23, 47.99s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


 50%|█████     | 2/4 [01:21<01:19, 39.52s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


 75%|███████▌  | 3/4 [01:42<00:30, 30.91s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


100%|██████████| 4/4 [01:56<00:00, 29.09s/it]


## Metric Results
The following metrics are calculated for each algorithm in the various splits.

We define the micro metric as the computation of all users and items in that
particular window.

We define the macro metric as the computation of all users and items across all
windows where the confusion matrix is first summed across all windows before
computing the metric.

In [4]:
evaluator.metric_results(level="macro")

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/streamsight/metrics/base.py:394: UserWarning: All predictions were off or the ground truth matrix was empty during compute of PrecisionK(timestamp_limit=1537776000,K=10).
  warn(UserWarning(f"All predictions were off or the ground truth matrix was empty during compute of {self.identifier}."))

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/streamsight/metrics/base.py:394: UserWarning: All predictions were off or the ground truth matrix was empty during compute of RecallK(timestamp_limit=1537776000,K=10).
  warn(UserWarning(f"All predictions were off or the ground truth matrix was empty during compute of {self.identifier}."))



macro_score  num_window
Algorithm                Metric                                
ItemKNNIncremental(K=10) PrecisionK_10     0.002122           4
                         RecallK_10        0.014975           4
ItemKNNRolling(K=10)     PrecisionK_10     0.000961           4
                         RecallK_10        0.007483           4
ItemKNNStatic(K=10)      PrecisionK_10     0.000961           4
                         RecallK_10        0.007483           4
Popularity(K=10)         PrecisionK_10     0.007126           4
                         RecallK_10        0.063441           4

In [5]:
evaluator.metric_results(level="micro")

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in scalar add
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)



micro_score  num_user
Algorithm                Metric                             
ItemKNNIncremental(K=10) PrecisionK_10    0.002233      7749
                         RecallK_10       0.016921      7749
ItemKNNRolling(K=10)     PrecisionK_10    0.001613      7749
                         RecallK_10        0.01186      7749
ItemKNNStatic(K=10)      PrecisionK_10    0.001613      7749
                         RecallK_10        0.01186      7749
Popularity(K=10)         PrecisionK_10    0.005291      7749
                         RecallK_10       0.043027      7749

In [6]:
evaluator.metric_results(level="window")

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/streamsight/metrics/base.py:394: UserWarning: All predictions were off or the ground truth matrix was empty during compute of PrecisionK(timestamp_limit=1537776000,K=10).
  warn(UserWarning(f"All predictions were off or the ground truth matrix was empty during compute of {self.identifier}."))

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/streamsight/metrics/base.py:394: UserWarning: All predictions were off or the ground truth matrix was empty during compute of RecallK(timestamp_limit=1537776000,K=10).
  warn(UserWarning(f"All predictions were off or the ground truth matrix was empty during compute of {self.identifier}."))



window_score  num_user
Algorithm                Timestamp    Metric                               
ItemKNNStatic(K=10)      t=1530000000 PrecisionK_10      0.002425      4289
                                      RecallK_10         0.017662      4289
                         t=1532592000 PrecisionK_10      0.000513      2532
                                      RecallK_10         0.003220      2532
                         t=1535184000 PrecisionK_10      0.000905       884
                                      RecallK_10         0.009050       884
                         t=1537776000 PrecisionK_10      0.000000        44
                                      RecallK_10         0.000000        44
ItemKNNRolling(K=10)     t=1530000000 PrecisionK_10      0.002425      4289
                                      RecallK_10         0.017662      4289
                         t=1532592000 PrecisionK_10      0.000513      2532
                                      RecallK_10         0.003220      2532
                         t=1535184000 PrecisionK_10      0.000905       884
                                      RecallK_10         0.009050       884
                         t=1537776000 PrecisionK_10      0.000000        44
                                      RecallK_10         0.000000        44
ItemKNNIncremental(K=10) t=1530000000 PrecisionK_10      0.002425      4289
                                      RecallK_10         0.017662      4289
                         t=1532592000 PrecisionK_10      0.002093      2532
                                      RecallK_10         0.016733      2532
                         t=1535184000 PrecisionK_10      0.001697       884
                                      RecallK_10         0.014140       884
                         t=1537776000 PrecisionK_10      0.002273        44
                                      RecallK_10         0.011364        44
Popularity(K=10)         t=1530000000 PrecisionK_10      0.000933      4289
                                      RecallK_10         0.007443      4289
                         t=1532592000 PrecisionK_10      0.011137      2532
                                      RecallK_10         0.085567      2532
                         t=1535184000 PrecisionK_10      0.009615       884
                                      RecallK_10         0.092572       884
                         t=1537776000 PrecisionK_10      0.006818        44
                                      RecallK_10         0.068182        44

To run the pipeline without ignoring unknown items and users, set ignore_unknown_item
and ignore_unknown_user to False. The default value for these parameters is True.

Below we can see the results of the evaluation for the different algorithms. The evaluation
results are shown for the micro and macro levels. Note that the number of users being
evaluated on would be now more than the case where the unknown items and users are ignored.

Note that there will be a need to load the dataset again as there would be modification
of the data in the setting object when the evaluator is executed.

In [7]:
from streamsight.datasets import AmazonMovieDataset
from streamsight.settings import SlidingWindowSetting

dataset = AmazonMovieDataset(use_default_filters=False)
data = dataset.load()
setting_sliding = SlidingWindowSetting(
    background_t=1530000000,
    window_size=60 * 60 * 24 * 30, # day times N
    n_seq_data=1,
    top_K=k
)
setting_sliding.split(data)


from streamsight.evaluators import EvaluatorPipelineBuilder

builder = EvaluatorPipelineBuilder(ignore_unknown_user=False,
                           ignore_unknown_item=False)
builder.add_setting(setting_sliding)
builder.set_metric_K(k)
builder.add_algorithm("ItemKNNIncremental", {"K": k})
builder.add_algorithm("Popularity", {"K": k})

builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
evaluator = builder.build()

evaluator.run()

INFO     - streamsight.datasets.base - AmazonMovieDataset is loading dataset...
INFO     - streamsight.datasets.base - AmazonMovieDataset dataset loaded - Took 12.5s


4it [00:01,  3.80it/s]                       

INFO     - streamsight.settings.sliding_window_setting - Finished split with window size 2592000 seconds. Number of splits: 4 in total.
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 1: Preparing the evaluator...



  0%|          | 0/4 [00:00<?, ?it/s]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


...

 50%|█████     | 2/4 [01:22<01:20, 40.08s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


 75%|███████▌  | 3/4 [01:40<00:29, 29.90s/it]

INFO     - streamsight.evaluators.evaluator_pipeline - Phase 2: Evaluating the algorithms...
INFO     - streamsight.evaluators.evaluator_pipeline - Phase 3: Releasing the data...


100%|██████████| 4/4 [01:52<00:00, 28.25s/it]


In [10]:
evaluator.metric_results(level="micro")

WARNING  - py.warnings - /Users/ngtzekean/Github/Streamsight/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in scalar add
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)



micro_score  num_user
Algorithm                Metric                             
ItemKNNIncremental(K=10) PrecisionK_10     0.00103     16794
                         RecallK_10       0.007778     16794
Popularity(K=10)         PrecisionK_10    0.006389     16794
                         RecallK_10       0.054041     16794

In [9]:
evaluator.metric_results(level="macro")

macro_score  num_window
Algorithm                Metric                                
ItemKNNIncremental(K=10) PrecisionK_10     0.000930           4
                         RecallK_10        0.006662           4
Popularity(K=10)         PrecisionK_10     0.008440           4
                         RecallK_10        0.076742           4